# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [32]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [33]:
city_data = pd.read_csv('../WeatherPy/city_data.csv',index_col=0)
# city_data = city_data.drop(columns=["Unnamed: 0"])
city_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,albany,42.60,-73.97,62.01,68,7,3.87,US,1596688754
1,skibbereen,51.55,-9.27,53.60,93,75,3.36,IE,1596688754
2,port moresby,-9.44,147.18,84.20,79,75,11.41,PG,1596688754
3,george,-33.96,22.46,46.40,81,75,12.75,ZA,1596688755
4,taveta,-3.40,37.67,60.80,93,75,6.64,KE,1596688755
...,...,...,...,...,...,...,...,...,...
531,buraydah,26.33,43.98,77.00,38,22,6.93,SA,1596688720
532,brazzaville,-4.27,15.28,68.00,94,20,6.46,CG,1596688617
533,khani,41.96,42.96,73.40,88,52,2.24,GE,1596688901
534,presque isle,46.68,-68.02,66.20,68,1,6.93,US,1596688901


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [34]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [35]:
# locations
locations = city_data[["Lat", "Lng"]]

# humidity
humidity = city_data["Humidity"].astype(float)
maxhumidity = humidity.max()


In [36]:
fig = gmaps.figure(zoom_level=4, center=(39.50, -98.35))

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [37]:
#A max temperature lower than 80 degrees but higher than 70 and no Cloudiness
ideal_city_df = city_data.loc[(city_data["Max Temp"] < 80) & (city_data["Max Temp"] > 70) & (city_data["Cloudiness"] == 0), :]
#Drop null values
ideal_city_df = ideal_city_df.dropna(how='any')
#rest indext
ideal_city_df.reset_index(inplace=True)
del ideal_city_df['index']

ideal_city_df.head()
len(ideal_city_df)

27

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [38]:
# Store into variable named hotel_df
hotel_df = pd.DataFrame(ideal_city_df, columns=["City", "Country", "Lat", "Lng"])
#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = pd.Series([])
hotel_df

<ipython-input-38-aadc9f5e48e2>:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  hotel_df["Hotel Name"] = pd.Series([])


,City,Country,Lat,Lng,Hotel Name
0,eregli,TR,37.51,34.05,NaN
1,sangar,RU,63.92,127.47,NaN
2,porto walter,BR,-8.27,-72.74,NaN
3,hervey bay,AU,-25.30,152.85,NaN
4,carnarvon,AU,-24.87,113.63,NaN
5,avarua,CK,-21.21,-159.78,NaN
6,abu samrah,SY,35.30,37.18,NaN
7,gat,IL,31.61,34.76,NaN
8,tselinnoye,RU,53.08,85.67,NaN
9,tatawin,TN,32.93,10.45,NaN


In [40]:
# Set parameters to search for hotels with 5000 meters.
hotellist = []

for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]['Lat']
    lng = hotel_df.loc[i]['Lng']

    params = {"location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key}
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
   # make request and print url
    name = requests.get(base_url, params=params)
    
    # convert to json
    name = name.json()

    try:
        hotellist.append(name['results'][0]['name'])
    except:
         hotellist.append("")
hotel_df["Hotel Name"] = hotellist
hotel_df = hotel_df.dropna(how='any') 
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,eregli,TR,37.51,34.05,Ereğli
1,sangar,RU,63.92,127.47,Sangar
2,porto walter,BR,-8.27,-72.74,Porto Walter
3,hervey bay,AU,-25.30,152.85,Hervey Bay
4,carnarvon,AU,-24.87,113.63,Carnarvon
5,avarua,CK,-21.21,-159.78,Avarua District
6,abu samrah,SY,35.30,37.18,Suruj
7,gat,IL,31.61,34.76,Kiryat Gat
8,tselinnoye,RU,53.08,85.67,Tselinnoye
9,tatawin,TN,32.93,10.45,Tataouine


In [41]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd> 
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [42]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))